In [ ]:
import numpy as np
import os
import cv2

In [31]:
def binaryMaskIOU(mask1, mask2):  
    mask1_area = np.count_nonzero(mask1 == 255)
    mask2_area = np.count_nonzero(mask2 == 255)
    intersection = np.count_nonzero(np.logical_and( mask1==255,  mask2==255 ))
    iou = intersection/(mask1_area+mask2_area-intersection)
    return iou

In [32]:
masks_path = '/home/aharris/shared/indianDB/masks_seg/'
masks = {}
for file in os.listdir(f'{masks_path}/masks/'):
    mask = cv2.imread(f'{masks_path}/masks/{file}')
    mask_name = file.split('.')[0]
    gray_mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    masks[f'IDRiD_{mask_name}'] = gray_mask

In [33]:
masks_OD = {}
for file in os.listdir(f'{masks_path}/optic_disc'):
    mask = cv2.imread(f'{masks_path}/optic_disc/{file}')
    mask_name = file.split('_')[1]
    gray_mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    th,binary = cv2.threshold(gray_mask, 1, 1, cv2.THRESH_BINARY_INV)
    masks_OD[f'IDRiD_{mask_name}'] = binary

### OCCLUSION

In [34]:
directory = '/home/aharris/shared/indianDB/occlusion/DR'
out_path = '/home/aharris/shared/indianDB/occlusion/IoU3'
if not os.path.exists(out_path):
    os.mkdir(out_path)

In [6]:
heatmaps = {}
for file in os.listdir(directory):
    if file.endswith('_map_smoothed.png'):
        filename = file.split('_')
        heatmap = cv2.imread(f'{directory}/{file}')
        gray_heatmap = cv2.cvtColor(heatmap,cv2.COLOR_BGR2GRAY)
        unique, counts = np.unique(gray_heatmap, return_counts=True)
        unique = list(unique)
        counts = list(counts)
        index = counts.index(np.max(counts))
        val = unique[index]
        if val > 190:
            val = val/2 
        heatmap2 = gray_heatmap * masks_OD[f'{filename[0]}_{filename[1]}']
        heatmap2  = np.where(heatmap2 == 0, val, heatmap2)
        heatmaps[f'{filename[0]}_{filename[1]}'] = heatmap2
        

In [7]:
results_occlusion = {}
for file in heatmaps:
    if file == 'IDRiD_43':
        continue
    else:
        print(file)
        results = {}
        img = heatmaps[file]
        thresholds = np.unique(img)
        cd = f'{out_path}/{file}'
        if not os.path.exists(cd):
            os.mkdir(cd) 
        heatmap_dim = img.shape
        resized_mask = cv2.resize(masks[file], (heatmap_dim[1],heatmap_dim[0]),interpolation=cv2.INTER_AREA)
        for t in thresholds:
            th,binary = cv2.threshold(img,t+5,255,cv2.THRESH_BINARY)
            cv2.imwrite(f'{cd}/{file}_{t}.jpg',binary)
            iou = binaryMaskIOU(masks[file],binary)
            results[t+5] = iou
        results_occlusion[file] = results
    

IDRiD_32
IDRiD_51
IDRiD_44
IDRiD_27
IDRiD_19
IDRiD_33
IDRiD_50
IDRiD_18
IDRiD_45
IDRiD_26
IDRiD_25
IDRiD_46
IDRiD_78
IDRiD_80
IDRiD_53
IDRiD_30
IDRiD_79
IDRiD_81
IDRiD_24
IDRiD_47
IDRiD_52
IDRiD_31
IDRiD_09
IDRiD_54
IDRiD_37
IDRiD_22
IDRiD_41
IDRiD_55
IDRiD_36
IDRiD_08
IDRiD_23
IDRiD_40
IDRiD_20
IDRiD_68
IDRiD_35
IDRiD_56
IDRiD_42
IDRiD_21
IDRiD_34
IDRiD_57
IDRiD_69
IDRiD_38
IDRiD_06
IDRiD_65
IDRiD_70
IDRiD_13
IDRiD_07
IDRiD_64
IDRiD_39
IDRiD_71
IDRiD_12
IDRiD_11
IDRiD_72
IDRiD_59
IDRiD_67
IDRiD_04
IDRiD_10
IDRiD_73
IDRiD_66
IDRiD_05
IDRiD_58
IDRiD_60
IDRiD_03
IDRiD_16
IDRiD_75
IDRiD_28
IDRiD_61
IDRiD_02
IDRiD_29
IDRiD_17
IDRiD_74
IDRiD_77
IDRiD_14
IDRiD_49
IDRiD_01
IDRiD_62
IDRiD_48
IDRiD_76
IDRiD_15
IDRiD_63


In [8]:
best_results = {}
for file in results_occlusion:
    maximum = max(results_occlusion[file], key=results_occlusion[file].get)
    best_results[file] = results_occlusion[file][maximum]

In [9]:
max(best_results, key=best_results.get)

'IDRiD_17'

In [10]:
best_results[max(best_results, key=best_results.get)]

0.2471582564591055

In [11]:
results_occlusion[max(best_results, key=best_results.get)]

{50.0: 0.18685975031992522,
 51.0: 0.18676046232188068,
 52.0: 0.18673499301755087,
 53.0: 0.18665210485975725,
 54.0: 0.18653070296357513,
 55.0: 0.1864972983171434,
 56.0: 0.1863836089473866,
 57.0: 0.18626901165596269,
 58.0: 0.18612692580488896,
 59.0: 0.18608383881060925,
 60.0: 0.1859042775742913,
 61.0: 0.18575492806559749,
 62.0: 0.1856235504560157,
 63.0: 0.1855511289098358,
 64.0: 0.18542556553533524,
 65.0: 0.18526856627541882,
 66.0: 0.185184353631438,
 67.0: 0.18513619424982933,
 68.0: 0.1851438226884197,
 69.0: 0.1853959632798285,
 70.0: 0.18538732912219313,
 71.0: 0.1853627901853903,
 72.0: 0.18567386471856098,
 73.0: 0.18567620265196164,
 74.0: 0.1856518496591883,
 75.0: 0.18565530539879005,
 76.0: 0.18592281596305915,
 77.0: 0.18592255818447098,
 78.0: 0.18591241825231455,
 79.0: 0.18590812072905835,
 80.0: 0.18624576195845002,
 81.0: 0.18625122362679403,
 82.0: 0.18623957682532205,
 83.0: 0.18624127091102136,
 84.0: 0.18666060281953656,
 85.0: 0.18667377380666095,
 86

In [12]:
mean =  np.mean(list(best_results.values()))
std =  np.std(list(best_results.values()))
print('mean + std optimal values:',mean,'+',std)

mean + std optimal values: 0.033770729496057285 + 0.0413012921792307


### Grad-CAM

In [21]:
directory = '/home/aharris/shared/indianDB/GradCAM/DR'
out_path = '/home/aharris/shared/indianDB/GradCAM/IoU3'
if not os.path.exists(out_path):
    os.mkdir(out_path)

In [23]:
heatmaps = {}
for file in os.listdir(directory):
    if file.endswith('_map_smoothed.jpg'):
        heatmap = cv2.imread(f'{directory}/{file}')
        gray = cv2.cvtColor(heatmap,cv2.COLOR_BGR2GRAY)
        filename = file.split('.')
        unique, counts = np.unique(gray, return_counts=True)
        unique = list(unique)
        counts = list(counts)
        index = counts.index(np.max(counts))
        val = unique[index]
        heatmap_dim = gray.shape
        resized_mask = cv2.resize(masks_OD[filename[0]], (heatmap_dim[1],heatmap_dim[0]),interpolation=cv2.INTER_AREA)
        heatmap2 = gray * resized_mask
        heatmap2  = np.where(heatmap2 == 0, val, heatmap2)
        heatmaps[f'{filename[0]}'] = heatmap2
        

In [24]:
results_gradcam = {}
for file in heatmaps:
    if file == 'IDRiD_43':
        continue
    else:
        print(file)
        results = {}
        img = heatmaps[file]
        thresholds = np.unique(img)
        cd = f'{out_path}/{file}'
        if not os.path.exists(cd):
            os.mkdir(cd) 
        heatmap_dim = img.shape
        resized_mask = cv2.resize(masks[file], (heatmap_dim[1],heatmap_dim[0]),interpolation=cv2.INTER_AREA)
        for t in thresholds:
            th,binary = cv2.threshold(img,t+5,255,cv2.THRESH_BINARY)
            cv2.imwrite(f'{cd}/{file}_{t}.jpg',binary)
            iou = binaryMaskIOU(resized_mask,binary)
            results[t+5] = iou
        results_gradcam[file] = results

IDRiD_62
IDRiD_09
IDRiD_13
IDRiD_78
IDRiD_73
IDRiD_18
IDRiD_80
IDRiD_02
IDRiD_69
IDRiD_40
IDRiD_31
IDRiD_51
IDRiD_20
IDRiD_26
IDRiD_57
IDRiD_37
IDRiD_46
IDRiD_04
IDRiD_75
IDRiD_15
IDRiD_64
IDRiD_03
IDRiD_68
IDRiD_72
IDRiD_81
IDRiD_19
IDRiD_12
IDRiD_79
IDRiD_63
IDRiD_08
IDRiD_21
IDRiD_50
IDRiD_30
IDRiD_41
IDRiD_47
IDRiD_36
IDRiD_56
IDRiD_27
IDRiD_65
IDRiD_14
IDRiD_74
IDRiD_05
IDRiD_38
IDRiD_53
IDRiD_49
IDRiD_22
IDRiD_29
IDRiD_42
IDRiD_58
IDRiD_33
IDRiD_71
IDRiD_60
IDRiD_11
IDRiD_17
IDRiD_66
IDRiD_06
IDRiD_77
IDRiD_35
IDRiD_44
IDRiD_24
IDRiD_55
IDRiD_59
IDRiD_32
IDRiD_28
IDRiD_48
IDRiD_23
IDRiD_39
IDRiD_52
IDRiD_10
IDRiD_61
IDRiD_01
IDRiD_70
IDRiD_76
IDRiD_07
IDRiD_67
IDRiD_16
IDRiD_54
IDRiD_25
IDRiD_45
IDRiD_34


In [25]:
best_results = {}
for file in results_gradcam:
    maximum = max(results_gradcam[file], key=results_gradcam[file].get)
    best_results[file] = results_gradcam[file][maximum]

In [26]:
max(best_results, key=best_results.get)

'IDRiD_17'

In [27]:
best_results[max(best_results, key=best_results.get)]

0.1745704358111353

In [28]:
results_gradcam[max(best_results, key=best_results.get)]

{20: 0.1745704358111353,
 21: 0.17271120325121442,
 22: 0.17046086688644496,
 23: 0.16825964025387985,
 24: 0.1658863838101201,
 25: 0.1633450902683908,
 26: 0.15993885533213548,
 27: 0.15749248610721486,
 28: 0.15562406007266189,
 29: 0.15412075233341677,
 30: 0.15391377419445207,
 31: 0.1532445335536181,
 32: 0.15311373644716555,
 33: 0.1529899541454438,
 34: 0.15119768844719517,
 35: 0.1509940954567476,
 36: 0.1492162519124952,
 37: 0.1489460947800479,
 38: 0.14852529672326953,
 39: 0.1483717667489164,
 40: 0.14821820177760536,
 41: 0.14759218672538396,
 42: 0.14736385487365128,
 43: 0.14653427376205785,
 44: 0.14634901235390574,
 45: 0.1462079653426111,
 46: 0.14621931858661977,
 47: 0.14599956149131085,
 48: 0.14571098329946577,
 49: 0.1455042773323974,
 50: 0.14519250666839698,
 51: 0.1450219189771071,
 52: 0.14485900239294802,
 53: 0.1444672099548895,
 54: 0.1442005153470965,
 55: 0.14370388583938673,
 56: 0.14346015902947948,
 57: 0.14310919362228086,
 58: 0.14287472651808109,


In [29]:
mean =  np.mean(list(best_results.values()))
std =  np.std(list(best_results.values()))
print('mean + std optimal values:',mean,'+',std)

mean + std optimal values: 0.01931667820525785 + 0.026509453285719027
